In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
import matplotlib.pyplot as plot
from matplotlib.pyplot import figure
import matplotlib.image as img
from datetime import datetime
import uuid,json,arcpy

# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
gis = GIS("https://www.arcgis.com", "ezhatcher", getpass())

········


In [2]:
#Define the function to access fields within a layer --> provides a pandas dataframe of the data table
def access_lyr(content,out_fld):
    base = content.layers
    lyr = base[0]
    topic_data = lyr.query(where="(CommunityID = '" + str(identifier) + "' OR CommunityName = '" + str(identifier) +"')" ,out_fields=out_fld)
    return topic_data.df

def getExtent(fromFeature, sp_ext_ref=102100):
    g = Geometry(fromFeature.geometry)
    if g.type.lower() == 'point':
        bg = Point({"x": g.x, "y": g.y, "spatialReference": {"wkid": sp_ext_ref}}).project_as(102100)
        gbuf = bg.buffer(2800) # assume meters, 1609.344 ~ 1 mile
        return gbuf.extent
    
    return g.extent

def getMapAction(fromFeature, forWebmap, sp_ref=102100, lyrNdx=0, oid_field='OBJECTID', label_field="OBJECTID", label_prefix="Feature Extent", add_popup=False):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}{1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    ext = getExtent(fromFeature, sp_ext_ref=sp_ref)
    
    # find the oid field
    #oid_field = [f for f in fromFeature.fields if 'objectid' in str(f.lower())][0] or 'ESRI_OID'
    #if len(oid_field) <= 0 or oid_field is None:
        #print('objectid not in field list, disabling popup')
        #add_popup = False
        
    wm = WebMap(forWebmap)
    layer = wm.layers[lyrNdx]
    
    lyr_lst = []
    main_lyr = wm.layers[lyrNdx].id
    for lyr in wm.layers:
        if lyr.id != main_lyr:
            lyr_lst.append(
            {
                "id": lyr.id,
                "visibility":False
            })
    
    #spatialRef = FeatureLayer(layer.url).properties.sourceSpatialReference.wkid ## Updated 9/4 EZH
    if add_popup and layer is None:
        print('layer not found, disabling popup')
        add_popup = False
    
    point_ft = Geometry(fromFeature.geometry)
    project = Point({"x":point_ft.x, "y": point_ft.y, "spatialReference": {"wkid":sp_ref}}).project_as(102100)
    
    popup_def = None
    if add_popup:
        popup_def = {
            "layerId": layer.id,
            "fieldName": oid_field, # we should probably get this dynamically and not hard code
            "fieldValue": fromFeature.attributes[oid_field], # same with this one
            "anchorPoint": {
                "x": project.x,
                "y": project.y,
                "spatialReference": {
                    "wkid": 102100
                }
            }
        }
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmap.id,
                "extent": {
                    "xmin": ext[0],
                    "ymin": ext[1],
                    "xmax": ext[2],
                    "ymax": ext[3],
                    "spatialReference":{
                        "wkid": 102100
                    }
                },
                "layers": lyr_lst,
                "popup": popup_def,
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<a data-storymaps="{0}" data-storymaps-type="media">{1}</a>'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

In [3]:
#Define the base template storymap using the item ID -- Hoonah Alaska
seedappItem = gis.content.get("2a51cb9cc9d54b57a52c67d8efdb5eba") #"2a51cb9cc9d54b57a52c67d8efdb5eba"
contentManager = arcgis.gis.ContentManager(gis)

In [4]:
#Use the base feature class that has the listing of communities for which a storymap will be created-- currently, it's the 
#boundaries feature service (on cloud), which also defines the map extent as the user scrolls through the panels
community_base = gis.content.get("8499c3bc4ff141588f5c9ab1db99b0fb")
find_comm = community_base.layers[0].query(where="NAME IS NOT NULL",out_fields="*").df.NAME
community_lst = []
for c in find_comm:
    community_lst.append(c)

In [ ]:
for c in set(community_lst):
    try:
        community = c
        print(str(community))
        clones = contentManager.clone_items(items=[seedappItem], folder="Municipality Storymaps", search_existing_items=False)
        clonedStorymap = storymap.JournalStoryMap(clones[0])
        commID = community_base.layers[0].query(where="NAME LIKE '%" + str(community) + "%'",out_fields="*").df.CommunityID[0]
        if str(commID) == 'None':
            identifier = community_base.layers[0].query(where="NAME LIKE '%" + str(community) + "%'",out_fields="*").df.NAME[0]
        else:
            identifier = commID
        clonedStorymap.save(title=str(community) +" Community Storymap", description="An interactive, printable overview of " + str(community) + " and basic community information.", tags="Community Storymap, Alaska DCRA," + str(community))
        print(community + " storymap cloned.")
        sections = clonedStorymap.properties["values"]["story"]["sections"]

        #Access the URL for the main photo from the Photo Link feature class
        try:
            photo_base = gis.content.get("6d9d437bbe2d48bfa87b4d9fca41f80d")
            find_photo = access_lyr(photo_base,'PhotoLink')
            main_pic = find_photo.PhotoLink[0]
        except:
            print('Issues with connecting available images to ' + str(community))

        #Identify/define the storymap panels
        panel_toc = sections[0]
        panel_ch = sections[1]
        panel_geog = sections[2]
        panel_demog = sections[3]
        panel_transport = sections[4]
        panel_edu = sections[5]
        panel_elect = sections[6]
        panel_econ = sections[7]
        panel_gensvs = sections[8]
        panel_ANCSA = sections[9]
        panel_contacts = sections[10]
        panel_bl = sections[11]

        panel_list = []
        panel_list.append(panel_demog)
        panel_list.append(panel_transport)
        panel_list.append(panel_edu)
        panel_list.append(panel_econ)
        panel_list.append(panel_contacts)


        #Modify the main title and photo in the main Table of Contents panel
        print("Updating table of contents and main picture...")
        panel_toc["title"] = '<span style="font-size:40px">' + str(community) + ', Alaska</span>'
        if str(main_pic) == 'None':
            panel_toc["media"]["image"]["url"] = "http://dcced.maps.arcgis.com/sharing/rest/content/items/34bf2ca84faa4b258712ed7a5422d0d5/data"
        else:
            panel_toc["media"]["image"]["url"] = main_pic
        clonedStorymap.save()

### TABLE OF CONTENTS ###
        pop_base = gis.content.get("64c4ed59e6c04b93a0e354f9e496ebb2")
        current_pop = pop_base.layers[0].query(where="(CommunityID = '" + str(identifier) + "' OR CommunityName = '" + str(identifier) + "') AND DataYear = 2017", out_fields = "*").df.Population[0]
        incrp_type = access_lyr(photo_base,'CommunityTypeName').CommunityTypeName[0]
        dt = community_base.layers[0].query(where="CommunityID = '" + str(identifier) +"'", out_fields = "*").df.INCRP_DATE[0]
        if str(dt) == 'None':
            panel_toc["content"] = '<style type="text/css">.blue {\n     display: inline-block;\n     background-color: #2c7bb6;\n     border-color: #2c7bb6 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><span style="font-weight: bold;">Current Population: ' + str(current_pop) + '</span></p><p>&nbsp;</p><p>Incorporation Type: ' + str(incrp_type) + '</p><p>&nbsp;</p>\n<p><span style="font-size:28px">Table of Contents</span><a data-storymaps="MJ-ACTION-1524782969520" data-storymaps-type="navigate"> </a></p>\n\n<p>&nbsp;</p>\n\n<p><a data-storymaps="MJ-ACTION-1524782969520" data-storymaps-type="navigate">Culture and History</a></p>\n     <p><a data-storymaps="MJ-ACTION-1537213657245" data-storymaps-type="navigate">Geography and Climate</a></p>\n     <p><span style="font-size:20px"><a class="peach" data-storymaps="MJ-ACTION-1524783369112" data-storymaps-type="navigate">Demographics</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1524784937326" data-storymaps-type="navigate">Transportation</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1524786044096" data-storymaps-type="navigate">Education</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1537978181320" data-storymaps-type="navigate">Municipal Officials</a></span></p><p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532548226455" data-storymaps-type="navigate">Economy</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1537213811429" data-storymaps-type="navigate">Business Licenses</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636740461" data-storymaps-type="navigate">General Services</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636735244" data-storymaps-type="navigate">ANCSA&nbsp;</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636729718" data-storymaps-type="navigate">Community Contacts</a></span></p>\n     <p>&nbsp;</p>\n     <p><span style="font-size:14px"><em>As you explore this storymap, you can click directly on map symbols and zoom around the map at any time to get more information about each item represented.</em></span></p>'
        else:
            incrp_date = str(dt)
            panel_toc["content"] = '<style type="text/css">.blue {\n     display: inline-block;\n     background-color: #2c7bb6;\n     border-color: #2c7bb6 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><span style="font-weight: bold;">Current Population: ' + str(current_pop) + '</span></p><p>&nbsp;</p><p>Incorporation Date: ' + str(incrp_date) + '</p><p>Incorporation Type: ' + str(incrp_type) + '</p><p>&nbsp;</p>\n<p><span style="font-size:28px">Table of Contents</span><a data-storymaps="MJ-ACTION-1524782969520" data-storymaps-type="navigate"> </a></p>\n\n<p>&nbsp;</p>\n\n<p><a data-storymaps="MJ-ACTION-1524782969520" data-storymaps-type="navigate">Culture and History</a></p>\n     <p><a data-storymaps="MJ-ACTION-1537213657245" data-storymaps-type="navigate">Geography and Climate</a></p>\n     <p><span style="font-size:20px"><a class="peach" data-storymaps="MJ-ACTION-1524783369112" data-storymaps-type="navigate">Demographics</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1524784937326" data-storymaps-type="navigate">Transportation</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1524786044096" data-storymaps-type="navigate">Education</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1537978181320" data-storymaps-type="navigate">Municipal Officials</a></span></p><p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532548226455" data-storymaps-type="navigate">Economy</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636740461" data-storymaps-type="navigate">General Services</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636735244" data-storymaps-type="navigate">ANCSA&nbsp;</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636729718" data-storymaps-type="navigate">Community Contacts</a></span></p>\n     <p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1537213811429" data-storymaps-type="navigate">Business Licenses</a></span></p><p>&nbsp;</p>\n     <p><span style="font-size:14px"><em>As you explore this storymap, you can click directly on map symbols and zoom around the map at any time to get more information about each item represented.</em></span></p>'

        clonedStorymap.save()
        
## CLONE THE WEBMAP ##
        #Access the webmap and clone the webmap/update map filters
        current_webmap = gis.content.get("bdbb441b8b5d4ef2bc1a519c44e49c6b")
        wm = arcgis.mapping.WebMap(webmapitem=current_webmap)
        wm_properties = {'title':'OpenData_Storymap_' + str(community),'snippet':'Test webmap clone','tags':['dcra','test']}
        wm.save(wm_properties,folder='Storymap Webmaps')
        
        #Find the new webmap and rename it
        new_map_search = gis.content.search('OpenData_Storymap_' + str(community), item_type="Web Map")
        webmap = new_map_search[0]
        new_webmap = webmap.id
        
        #Define the webmap item to access and filter layers
        upd_webmap = gis.content.get(new_webmap)
        wm_instory = WebMap(webmapitem=upd_webmap)
        query = "(CommunityName = '" + str(identifier) + "' OR CommunityID = '" + str(identifier) +"')"

        #Query to use throughout panel updates when schema matches standard CDO schema (CommunityName and CommunityID)
        query = "(CommunityName = '" + str(identifier) + "' OR CommunityID = '" + str(identifier) +"')"

#UPDATE WEBMAP LAYER FILTERS
        #Find regions for regional web map filters
        region_lyr = gis.content.get("5522cf5f5e5d4d52abe6e7df47d5d241").layers[0].query(where="Community_Name = '" + str(community) + "'", out_fields = "*")
        ANCSA_rgn = region_lyr.df.ANCSA_Regional_Corporation[0]
        econ_rgn = region_lyr.df.Economic_Region[0]
        sch_distr = region_lyr.df.School_District[0]
        borough = region_lyr.df.Borough_Census_Area[0]

        filter_ID = "CommunityID = '" + str(commID) + "'"

        #Taxes
        wm_instory.layers[0]["layerDefinition"]["definitionExpression"] = "(CommunityID = '" + str(commID) +"') AND (RevenueYear = 2016)"

        #Income and Poverty
        wm_instory.layers[1]["layerDefinition"]["definitionExpression"] = filter_ID

        #Municipality Contacts
        wm_instory.layers[2]["layerDefinition"]["definitionExpression"] = filter_ID

        #School Enrollment
        wm_instory.layers[3]["layerDefinition"]["definitionExpression"] = filter_ID

        #DCCED Certified Population
        wm_instory.layers[4]["layerDefinition"]["definitionExpression"] = filter_ID

        #Ferries
        wm_instory.layers[5]["layerDefinition"]["definitionExpression"] = filter_ID

        #Healthcare facilities
        wm_instory.layers[6]["layerDefinition"]["definitionExpression"] = filter_ID

        #Community Regions Overview
        wm_instory.layers[7]["layerDefinition"]["definitionExpression"] = filter_ID

        #Alaska Road System
        #wm_instory.layers[8]

        #Boro_REAAs
        #wm_instory.layers[9]

        #Alaska Airports
        wm_instory.layers[10]["layerDefinition"]["definitionExpression"] = filter_ID

        #City Bounding Area
        wm_instory.layers[11]["layerDefinition"]["definitionExpression"] = "NAME = '" + str(community) + "'"

        #ANCSA Regional Corporation
        wm_instory.layers[12]["layerDefinition"]["definitionExpression"] = "NAT_CORP = '" + str(ANCSA_rgn) + "'"

        #Economic Region Employment and Wages
        wm_instory.layers[13]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(econ_rgn) + "'"

        #School Disricts
        wm_instory.layers[16]["layers"][0]["layerDefinition"]["definitionExpression"] = "School_District = '" + str(sch_distr) + "'"

        wm_instory.update()
        print(str(community) + " webmap id: " + str(new_webmap))


### UPDATE STORYMAP PANELS WITH CITY EXTENT ###
        #Set up zoom scale (x max and min, y max and min) for map actions 
        bounds_ft = gis.content.get("8499c3bc4ff141588f5c9ab1db99b0fb")
        ft_lyr = FeatureLayer(bounds_ft)
        boundary_lyr = ft_lyr.url.layers[0].query(where="NAME = '" + str(community) + "' OR CommunityID = '" + str(commID) + "'", out_fields="*").features[0].geometry
        bounds_ext = Polygon(boundary_lyr).extent
        c_ext = Geometry({
            'xmin':bounds_ext[0],
            'ymin':bounds_ext[1],
            'xmax':bounds_ext[2],
            'ymax':bounds_ext[3],
            'spatialReference':{
                'wkid':102006
            }
        }).project_as(3857)

        #Update primary zoom scale of each panel based on the minimum bounding envelope feature service
        # item id = "8499c3bc4ff141588f5c9ab1db99b0fb" using previously defined zoom scale
        print("Updating panel extents...")
        for p in panel_list:
            p["media"]["webmap"]["id"] = new_webmap
            ext1 = p["media"]["webmap"]["extent"]
            ext1["xmin"] = c_ext.xmin-50
            ext1["xmax"] = c_ext.xmax-50
            ext1["ymin"] = c_ext.ymin+50
            ext1["ymax"] = c_ext.ymax+50
            clonedStorymap.save()
        print("Panel extents updated.")
### CULTURE HISTORY PANEL ###
        #Access the culture and history from the culture/history feature class (IF THE PANEL ORDER CHANGES, MAKE SURE INDEX STILL APPLIES)
        panel_ch["media"]["webmap"]["id"] = new_webmap
        print("Updating culture and history panel content...")
        ch_ft = gis.content.get("3f99cf21a0da42c4bf8b15632e958df9")
        culture_lookup = access_lyr(ch_ft,'CommunityCulture')
        culture = culture_lookup.CommunityCulture[0]
        history_lookup = access_lyr(ch_ft,'CommunityHistory')
        history = history_lookup.CommunityHistory[0]

        #Populate the panel with the new culture history info
        panel_ch["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;<p>'+ history + '&nbsp;</p>\n\n<p>\n\n<p>&nbsp;<p>'+ culture + '&nbsp;</p>\n'
        panel_ch["media"]["webmap"]["altText"] = 'Map showing the location of ' + str(community) + ', Alaska in the context of the state and borough/REAA boundaries.'
        clonedStorymap.save()
        print("Culture and history panel completed.")
        
### GEOGRAPHY AND CLIMATE ###
        print("Updating climate and geography panel...")
        geog_base = gis.content.get("85b622ae19b04c7caa37c193b20fc4f3")
        geog_lookup = access_lyr(geog_base, 'Location, Climate, Latitude, Longitude, Sq_Mi_Land, Sq_Mi_Water')
        panel_geog["content"] = "<style type='text/css'>.dcra-blue {\n display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style><p><a class='dcra-blue' data-storymaps='MJ-ACTION-1537213609501' data-storymaps-type='navigate'>Back to Table of Contents</a></p><p>&nbsp;</p><p><b>Latitude, Longitude: </b>" + str(geog_lookup.Latitude[0]) + ', ' + str(geog_lookup.Longitude[0]) + '</p><p>&nbsp;</p><p>' + str(geog_lookup.Location[0]) + '</p><p>&nbsp;</p><p><b>Climate: </b>' + str(geog_lookup.Climate[0]) + '</p><p>&nbsp;</p><p><b>Square Miles of Land: </b>' + str(geog_lookup.Sq_Mi_Land[0]) + '</p><p><b>Square Miles of Water: </b>' + str(geog_lookup.Sq_Mi_Water[0]) + '</p>'

        clonedStorymap.save()
        print("Geography and climate info posted to geog/climate panel.")

        #Update the webmap in the panel to new webmap
        panel_geog["media"]["webmap"]["id"] = new_webmap

        #Update popup extents
        pt_ft = Geometry(region_lyr.features[0].geometry)
        project_geog = Point({"x":pt_ft.x, "y": pt_ft.y, "spatialReference": {"wkid":4326}}).project_as(102100)
        panel_geog["media"]["webmap"]["popup"]["fieldName"] = 'OBJECTID'
        panel_geog["media"]["webmap"]["popup"]["fieldValue"] = int(region_lyr.df.OBJECTID[0])
        panel_geog["media"]["webmap"]["popup"]["anchorPoint"]["x"] = project_geog.x
        panel_geog["media"]["webmap"]["popup"]["anchorPoint"]["y"] = project_geog.y

        clonedStorymap.save()

### DEMOGRAPHICS PANEL ###
        #Update the zoom extent for the demographics panel map action (tied to 'Historic Census' button)
        panel_demog["media"]["webmap"]["id"] = new_webmap
        demog_ext = panel_demog["media"]["webmap"]["extent"]
        demog_ext["xmin"] = c_ext.xmin
        demog_ext["xmax"] = c_ext.xmax
        demog_ext["ymin"] = c_ext.ymin
        demog_ext["ymax"] = c_ext.ymax
        print("Demographics panel extent updated.")
        #Update the main DCCED popup
        #demog_popup = panel_demog["contentActions"][1]["media"]["webmap"]["popup"]

        panel_demog["media"]["webmap"]["altText"] = 'Map showing aerial imagery of ' + str(community) + ', Alaska and a pop-up box displaying current and historic DCCED certified population counts from 2011 to present.'


        #Update popup extents
        print("Updating demographics popup...")
        dcced_pop = FeatureLayer('https://maps.commerce.alaska.gov/server/rest/services/Demographics/Population_DCCED_View/MapServer/0')
        pop_lyr = dcced_pop.query(where=filter_ID, out_fields = "*")
        point_ft = Geometry(pop_lyr.features[0].geometry)
        project_dcced = Point({"x":point_ft.x, "y": point_ft.y, "spatialReference": {"wkid":4326}}).project_as(102100)
        panel_demog["media"]["webmap"]["popup"]["fieldName"] = 'OBJECTID'
        panel_demog["media"]["webmap"]["popup"]["fieldValue"] = int(pop_lyr.df.OBJECTID[0])
        panel_demog["media"]["webmap"]["popup"]["anchorPoint"]["x"] = project_dcced.x
        panel_demog["media"]["webmap"]["popup"]["anchorPoint"]["y"] = project_dcced.y

        print("Demographics popup updated. Anchor point = (" + str(project_dcced.x) + ", " + str(project_dcced.y) + ").")

        clonedStorymap.save()


        #Historical census table for demographics panel
        print("Locating census data for " + str(community) +"...")
        try:
            census_base = gis.content.get("b475ec701f25448aa6e03cb16d94b6f0") #Census locations all locations
            pop_df = access_lyr(census_base,'*')
        except:
            print("QUERY ERROR: Cannot find " + str(community) + "in census feature.")
        census_tbl = ["<table border='1'><th>Census Year</th><th>Population</th><tbody>"]
        for y in pop_df.CensusYear:
            year_query = census_base.layers[0].query(where="(CommunityID = '" + str(identifier) + "') AND CensusYear = '" + str(y) + "'",out_fields="*")
            pop = year_query.df.CensusPopulation[0]
            census_tbl.append('<tr><td>' + str(y) + '</td><td>' + str(pop) + '</td></tr>')
        census_tbl.append('</tbody></table>')
        embed_census_tbl = (' '.join(census_tbl))
        print("Census table built.")
        
        #Race composition of community as table for demographics panel
        race_base = gis.content.get("7c39a49137ff44598b2ee4f9e444eff5")
        race_data = access_lyr(race_base,'*')
        if str(race_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
            embed_poprace = ('Data on race is unavailable for ' + str(community) + '.')
        else:
            race_pop = race_data.TotalPop[0]
            print("Building race table...")
            demog_race = ["<table border = '1'><tr><th width='70%'>Race</th><th width='30%'>Percent of Population</th></tr><tbody>",
                          "<tr><td>" + 'American Indian or AK Native' + "</td><td>" + str(round(int(race_data.AmInd_AKNat[0])/race_pop*100,2)) + "%</td></tr>",
                          "<tr><td>" + 'Asian' + "</td><td>" + str(round(int(race_data.Asian[0])/race_pop*100,2)) + "%</td></tr>",
                          "<tr><td>" + 'Black or African American' + "</td><td>" + str(round(int(race_data.Black_AfricanAmerican[0])/race_pop*100,2)) + "%</td></tr>",
                          "<tr><td>" + 'Native Hawaiian or Pacific Islander' + "</td><td>" + str(round(int(race_data.NatHI_PacIsland[0])/race_pop*100,2)) + "%</td></tr>",
                          "<tr><td>" + 'White' + "</td><td>" + str(round(int(race_data.White[0])/race_pop*100,2)) + "%</td></tr>",
                          "<tr><td>" + 'Other Race' + "</td><td>" + str(round(int(race_data.OtherRace[0])/race_pop*100,2)) + "%</td></tr>",
                          "<tr><td>" + 'Two or More Races' + "</td><td>" + str(round(int(race_data.TwoOrMore[0])/race_pop*100,2)) + "%</td></tr>"]

            embed_poprace = (' '.join(demog_race))

        #Age composition
        age_base = gis.content.get("62c257b7b48b456c92fd343337e45421")
        age_data = access_lyr(age_base,'*')
        if str(age_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
            print("No age data for " + str(community))
            embed_popage = ('Data on age is unavailable for ' + str(community) + '.')
        else:
            total_pop_acs = int(age_data.estimate_totalpop[0])
            print("Building age table...")
            demog_age = ["<table border = '1'><tr><th width='70%'>Age</th><th width='30%'>Percent of Population</th></tr><tbody>",
                          "<tr><td>" + 'Under 5 years of age' + "</td><td>" + str(round(int(age_data.estimate_under5__estimate_under[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '5 to 9' + "</td><td>" + str(round(int(age_data.estimate5_9[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '10 to 14' + "</td><td>" + str(round(int(age_data.estimate10_14[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '15 to 19' + "</td><td>" + str(round(int(age_data.estimate15_19[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '20 to 24' + "</td><td>" + str(round(int(age_data.estimate20_24[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '25 to 34' + "</td><td>" + str(round(int(age_data.estimate25_34[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '35 to 44' + "</td><td>" + str(round(int(age_data.estimate35_44[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '45 to 54' + "</td><td>" + str(round(int(age_data.estimate45_54[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '55 to 59' + "</td><td>" + str(round(int(age_data.estimate55_59[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '60 to 64' + "</td><td>" + str(round(int(age_data.estimate60_64[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '65 to 74' + "</td><td>" + str(round(int(age_data.estimate65_74[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + '75 to 84' + "</td><td>" + str(round(int(age_data.estimate75_84[0])/total_pop_acs*100,2)) + "%</td></tr>",
                          "<tr><td>" + 'Over 85 years of age' + "</td><td>" + str(round(int(age_data.estimate85plus[0])/total_pop_acs,2)) + "%</td></tr>",]

            embed_popage = (' '.join(demog_age))

        #Sex composition
        m_f_base = gis.content.get("daec36774ad240829d587d8c8fc3dd54")
        sex_data = access_lyr(m_f_base,'*')
        if str(sex_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
            print("No sex data for " + str(community))
            embed_popsex = ('Data on sex is unavailable for ' + str(community) + '.')
        else:
            print("Building sex table...")
            demog_sex = ["<table border = '1'><tr><th width='70%'>Sex</th><th width='30%'>Percent of Population</th></tr><tbody>",
                         "<tr><td>" + 'Male' + "</td><td>" + str(round(int(sex_data.female_estimate[0])/total_pop_acs*100,2)) + "%</td></tr>",
                         "<tr><td>" + 'Female' + "</td><td>" + str(round(int(sex_data.male_estimate[0])/total_pop_acs*100,2)) + "%</td></tr>"]

            embed_popsex = (' '.join(demog_sex))
            panel_demog["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783883308" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>Every June the Commissioner of the Department of Commerce Community Economic Development certifies community population figures. The most recent figures are displayed in the map popup.</p>\n\n<p>&nbsp;</p>\n\n<p> Historic census population counts below for the community provide a century\'s worth of change in this community:</p>\n\n<p><div>' + str(embed_census_tbl) + '</div><p>&nbsp;</p><div><p><b> Current Population by Race:</b></p><p>' + str(embed_poprace) +'</p></table></div><p>&nbsp;</p><div><p><b> Current Population by Age:</b>' + str(embed_popage) + '</table></p></div>\n\n<p>&nbsp;</p><div><p><b> Current Population by Sex*:</b>' + str(embed_popsex) + '</table></p></div>\n\n<p>&nbsp;</p><p><i>*From the US Census Bureau on sex versus gender: "In general discussions, the concept of gender is often confused with the concept of sex, and the terms are used interchangeably. The meanings of these two concepts are not the same: sex is based on the biological attributes of men and women (chromosomes, anatomy, hormones), while gender is a social construction whereby a society or culture assigns certain tendencies or behaviors to the labels of masculine or feminine." </i></p><p>&nbsp;</p><p>More information about population trends can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=577407acfbc6433389006d099cb25971" target="_blank">here.&nbsp;</a></p>\n'
            print("Demographics panel updated.")

        clonedStorymap.save()

###TRANSPORTATION PANEL###
        #Update panel text using transportation overview feature class (item id = "2076ce8b6744430880e1c8ab923cd720")
        find_text = gis.content.get("4b1dc0106a7c45dab6bfcdcef18bb84d")
        access_text = find_text.layers[0].query(where="CommunityName = '" + str(identifier) + "' OR CommunityID_1 = '" + str(identifier) +"'", out_fields = "*").df

        #Create airport listing table in panel
        act_no = len(panel_transport["contentActions"])
        panel_transport["contentActions"][1:act_no] = []
        arpt_base = gis.content.get("5728f130d89e4d5d97278b4f63d12caa")
        arpt_data = access_lyr(arpt_base,'*')
        if str(arpt_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
            print("There are no public use airports in " + str(community))
            panel_transport["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n}\n</style>\n<style type="text/css">.btn-orange {\n    display: inline-block;\n    background-color: #f0ad4e;\n    border-color: #f0ad4e !important;\n    color: #fff !important;\n    padding: 0px 2px;\n    border-radius: 12px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524784108786" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>' + access_text.Description[0] + '</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>More detailed information about Transportation in Alaska can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=3de3ebccbb6b4ba8a9c64609e96199ba" target="_blank"><strong>here</strong></a>.&nbsp;</p>\n'
        else:
            def airport_lst(airports):
                arpt_listing = ["<table border = '1'><tr><th width='70%'>Airport</th><th width='30%'>Airport Code</th></tr><tbody>"]
                for a in airports.EntityName:
                    airport_data = arpt_base.layers[0].query(where="EntityName = '" + str(a) + "'",out_fields='EntityName, AirportCode')
                    arpt_code = airport_data.df.AirportCode[0]
                    ma = getMapAction(airport_data.features[0], upd_webmap, sp_ref = 102100, lyrNdx=10, oid_field='OBJECTID_1', label_field='EntityName',label_prefix="", add_popup=True)
                    panel_transport["contentActions"].append(ma["map_action"])
                    arpt_listing.append("<tr><td>" + ma['content_link'] + "</td><td>" + str(arpt_code) + "</td><td>")
                arpt_listing.append("</tbody></table>")
                return(' '.join(arpt_listing))
            airport_lst(arpt_data)

            panel_transport["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n}\n</style>\n<style type="text/css">.btn-orange {\n    display: inline-block;\n    background-color: #f0ad4e;\n    border-color: #f0ad4e !important;\n    color: #fff !important;\n    padding: 0px 2px;\n    border-radius: 12px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524784108786" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>' + str(access_text.Description[0]) + '</p>\n\n<p>&nbsp;</p><p>Click on the icons in the map to see specific information about each airport or ferry terminal.</p>\n\n<p>&nbsp;</p>\n\n<p>' + str(airport_lst(arpt_data)) + '</p><p>&nbsp;</p>\n\n<p>More detailed information about Transportation in Alaska can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=3de3ebccbb6b4ba8a9c64609e96199ba" target="_blank"><strong>here</strong></a>.&nbsp;</p>\n'
            clonedStorymap.save()
            print("Transportation table posted to panel.")

####EDUCATION PANEL####
        act_no = len(panel_edu["contentActions"])
        panel_edu["contentActions"][1:act_no] = []
        ak_schools = gis.content.get("42a9b3ab0392444aa25cae39d19ce221")
        community_sch_lst = ak_schools.layers[0].query(where="CommunityN = '" + str(community) + "'", out_fields = "*").df

        ##Check to see if there are schools. If not, post statement to panel that there are no open schools in community. 
        ##otherwise, create a schools table.
        output = str(community_sch_lst)
        if output == 'Empty DataFrame\nColumns: []\nIndex: []':
            print("There are no schools in " + str(community))
            panel_edu["content"] = '<p>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>There are no schools currently open in' + str(community) + '</p><p>&nbsp;</p>\n\n<p>For more information about Alaska schools and education, click <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=cff461c2397d4c66ae2c28dbfb62319f" target="_blank">here</a>.</p>\n'
        else:
            schools = community_sch_lst.EntityName
            def school_lst():
                school_list = ["<table border = '1'><tr><th width='45%'>School</th><th width='45%'>Address</th><th width='10%'>Zip Code</th></tr><tbody>"]
                for s in schools:
                    #school_data = ak_schools.layers[0].query(where="EntityName = '" + str(e) + "'",out_fields= 'EntityName,PhysicalAd,PhysicalCi,PhysicalZi')
                    school_data = ak_schools.layers[0].query(where="EntityName = '" + str(s.replace("'","''")) + "'",out_fields='EntityName,PhysicalAd,PhysicalCi,PhysicalZi')
                    address = school_data.df.PhysicalAd[0]
                    city = school_data.df.PhysicalCi[0]
                    zipcode = school_data.df.PhysicalZi[0]
                    school_list.append("<tr><td>" + str(s) + "</td><td>" + address + "</td><td>" + zipcode + '</td></tr>')
                school_list.append("</tbody></table>")
                return(' '.join(school_list))

            school_lst()

            #Update panel schools listing
            panel_edu["content"] =  '<p>&nbsp;</p>\n\n<p>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>' + str(community) + " schools are part of " + str(sch_distr) + ". </p>\n\n<p>&nbsp;</p><p><i>Click the school icons directly in the map to see enrollment data from 2011 to present for each school. </i></p><p>\n\n<p>&nbsp;</p><p>" + str(school_lst()) + '</p><p>&nbsp;</p>\n\n<p>For more information about Alaska schools and education, click <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=cff461c2397d4c66ae2c28dbfb62319f" target="_blank">here</a>.</p>\n'
            clonedStorymap.save()
            print("School table posted to the education panel.")

### ECONOMY PANEL EXTENT ###
        #Set up zoom scale (x max and min, y max and min)
        econ_ft = FeatureLayer('https://maps.commerce.alaska.gov/arcgis/rest/services/Economics_Related/Economics_Employment_Wages/MapServer/36')
        boundary_lyr = econ_ft.query(where="CommunityName LIKE '%" + str(econ_rgn) + "%'", out_fields="*").features[0].geometry
        econ_base_ext = Polygon(boundary_lyr).extent
        e_ext = Geometry({
            'xmin':econ_base_ext[0],
            'ymin':econ_base_ext[1],
            'xmax':econ_base_ext[2],
            'ymax':econ_base_ext[3],
            'spatialReference':{
                'wkid':102006
            }
        }).project_as(3857)

        econ_ext = panel_econ["media"]["webmap"]["extent"]
        econ_ext["xmin"] = e_ext.xmin
        econ_ext["xmax"] = e_ext.xmax
        econ_ext["ymin"] = e_ext.ymin
        econ_ext["ymax"] = e_ext.ymax
        clonedStorymap.save()

        
### ECONOMY PANEL ###
        #Employment rate from employment and wages data
        emp_wgs_base = gis.content.get("601a1796d41d485992a18e2974845532").layers[0]
        emp_wgs_data = emp_wgs_base.query(where=str(query) + " AND DataYear = 2016", out_fields="*").df
        emp_rate = round(float(emp_wgs_data.ResidentsEmployed[0]/emp_wgs_data.ResidentsAge16AndOver * 100),1)
        emp_rate

        #Taxes data
        tax_base = gis.content.get("70a364eae3514531a280e4696b54c6e9").layers[0]
        tax_data = tax_base.query(where="CommunityName = '" + str(community) + "' AND RevenueYear = 2016", out_fields = "*")
        sales_tax = tax_data.df.SalesTaxPercentage[0]
        prop_tax = tax_data.df.PropertyTaxMills[0]
        other_tax = tax_data.df.OtherTaxes[0]
        prop_tax_rev = tax_data.df.PropertyTaxRevenue[0]
        sales_tax_rev = tax_data.df.SalesTaxRevenue[0]
        total_tax_rev = tax_data.df.TotalTaxRevenue[0]
        tax_table = ['<table><tr><th width="70%"></th><th width="30%"></th></tr>',
                     '<tr><td>Property Tax Mills:</td><td>' + str(prop_tax) + '</td></tr>',
                    '<tr><td>Sales Tax Rate: </td><td>' + str(sales_tax) + '</td></tr>',
                    '<tr><td>Other Taxes: </td><td>' + str(sales_tax) + '</td></tr>',
                    '<tr><td>Total Tax Revenue: </td><td> $' + str(total_tax_rev) + '</td></tr>',
                    '<tr><td>Sales Tax Revenue: </td><td> $' + str(sales_tax_rev) + '</td></tr>']
        embed_tax_tbl = ' '.join(tax_table)

        #Income and poverty data
        inc_pov_base = gis.content.get("ad27d89db59744b89766e8131af0c13d")
        inc_pov_data = inc_pov_base.layers[0].query(where=str(query) + " AND IsMostRecent = 1", out_fields = "*").df
        hs_inc = inc_pov_data.MedianHouseholdIncome[0]
        fm_inc = inc_pov_data.MedianFamilyIncome[0]
        poverty = inc_pov_data.PersonsBelowPoveryLevel[0]
        poverty125 = inc_pov_data.PersonsBelow125PercentOfPovLev[0]
        inc_pov_tbl = ['<table><tr><th width="70%"></th><th width="30%"></th></tr>',
                     '<tr><td>Median Household Income:</td><td> $' + str(hs_inc) + '</td></tr>',
                    '<tr><td>Median Family Income: </td><td> $' + str(fm_inc) + '</td></tr>',
                    '<tr><td>Persons Below Poverty: </td><td>' + str(poverty) + '</td></tr>',
                    '<tr><td>Persons Below 125% of the Poverty Level: </td><td> ' + str(poverty125) + '</td></tr></table>']
        embed_IncPov_tbl = ' '.join(inc_pov_tbl)

        panel_econ["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p><b><u>Employment Rate: </u></b>' + str(emp_rate) + '%</p><p>&nbsp;</p>\n\n<div><p><b><u>Taxes: </u></b> ' + str(embed_tax_tbl) + '</table></p></div><p>&nbsp;</p><p><b><u>Income and Poverty:</u></b>' + str(embed_IncPov_tbl) + '</p>\n\n<p>&nbsp;</p><p>More information can be found in the <a href="http://DCCED.maps.arcgis.com/apps/MapJournal/index.html?appid=bb631449256346db81bc026339f0e60c" target="_blank">Alaska Taxable.</a></p>\n'

        clonedStorymap.save()
        print("Economic data posted to economy panel.")

### MUNICIPAL OFFICIALS PANEL EXTENTS AND FILTERS ###                      
#Define method for embedding officials data
        def officials_tbl(base):
            test = base.query(where=query, out_fields="*").df
            if str(test) == 'Empty DataFrame\nColumns: []\nIndex: []':
                print("There are no officials in" + str(base))
            else:
                officials = base.query(where=query, out_fields="*").df.OfficialName
                offic_tbl = ['<table border="1"><th width="60%">Official Name</th><th width="40%">Term End Date</th><tbody>']
                for o in officials:
                    official = base.query(where="OfficialName = '" + str(o) + "'", out_fields="*").df.OfficialName[0]
                    o_term = str(base.query(where=query, out_fields="*").df.TermEnds[0])
                if o_term == 'None':
                    o_end_dt = 'no data'
                else:
                    o_term_unix = str(datetime.utcfromtimestamp(int(o_term[0:10])))
                    o_end_dt = datetime.strftime(datetime.strptime(o_term_unix, "%Y-%m-%d %H:%M:%S"), '%m/%d/%Y')
                offic_tbl.append('<tr><td>' + str(official) + '</td><td>' + str(o_end_dt) + '</td></tr>')
                offic_tbl.append('</tbody></table>')
                return(' '.join(offic_tbl))

        #Get house and senate districts for the community
        print("Updating municipal officials content...")
        try:
            election_base = gis.content.get("4e550123a73448698a63c204aedf146b")
            filter_by_location = arcgis.geometry.filters.intersects(c_ext, sr=102100)
            precincts = election_base.layers[0].query(out_fields="*", geometry_filter=filter_by_location).df.NAME
        except:
            print("QUERY ERROR: Cannot find " + str(community) + " in election districts feature service.")
        precinct1 = election_base.layers[0].query(out_fields="*", geometry_filter=filter_by_location).df.NAME[0]
        house_district = precinct1[0:2]
        #Query to filter data by house district
        house_filter = "HouseDistrict = '" + str(house_district) + "'"

        #Update house and senate distrct webmap layer to only show the house and senate district of the the community
        wm_instory.layers[14]["layerDefinition"]["definitionExpression"] = house_filter
        wm_instory.update()

        house_ft = election_base.layers[1].query(where=house_filter,out_fields="*").features[0]
        house_buffer = Polygon(house_ft.geometry).buffer(15000)
        house_ext = house_buffer.extent
        elect_ext = Geometry({
            'xmin':house_ext[0],
            'ymin':house_ext[1],
            'xmax':house_ext[2],
            'ymax':house_ext[3],
            'spatialReference':{
                'wkid':102100
            }
        })


        #Use the house district to find the senate district
        senate_dist = election_base.layers[1].query(where=house_filter,out_fields="*").df.SenateDistrict[0]

        #Filter the map to only the precincts that fall in the district
        #filter_by_district = 
        #wm_instory.layers[15]

        #City Council
        cc_base = gis.content.get("ec5f29ac99aa4f84939d1d4dcee82568").layers[0]
        embed_cc_tbl = officials_tbl(cc_base)
        print("City council table built.")

        #School Board
        #sb_base = gis.content.get("36823739e8f54b979c9c559d20ba6386").layers[0]
        #embed_sb_tbl = official_tbl(sb_base)

        #Borough Assembly
        boro_base = gis.content.get("3631c17312c34b6f8dfb9ccfc2ae4b5c").layers[0]
        boro_query = boro_base.query(where="EntityName = '" + str(borough) + "'", out_fields="*").df
        if str(boro_query) == 'Empty DataFrame\nColumns: []\nIndex: []': 
            print("No assembly data will be added.")
            embed_boro_tbl = "No data."
        else:
            boro_officials = boro_query.OfficialName
            boro_tbl =['<table border=1><th><b>Borough Assembly Member</b></th><th><b>Term End Date</b></th><tbody>']
            for b in boro_officials:
                boro_data = boro_base.query(where="OfficialName = '" + str(b) + "'", out_fields = "*")
                boro_dt = str(boro_data.df.TermEnds[0])
                # Time comes in as a UNIX timestamp, convert to a simple mm/dd/yyyy format
                if boro_dt == 'None':
                    boro_term_dt = 'no data'
                else:
                    boro_term_unix = str(datetime.utcfromtimestamp(int(boro_dt[0:10])))
                    boro_term_dt = datetime.strftime(datetime.strptime(boro_term_unix, "%Y-%m-%d %H:%M:%S"), '%m/%d/%Y')
                boro_tbl.append("<tr><td>" + str(b) + "</td><td>" + str(boro_term_dt) + "</td></tr>")
            boro_tbl.append("</tbody></table>")
            embed_boro_tbl = ' '.join(boro_tbl)

        #Mayor
        mayor_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/8')
        mayor = mayor_base.query(where="CommunityID = '" + str(commID) +"'", out_fields="*").df.OfficialName[0]
        mayor_end = str(mayor_base.query(where="CommunityID = '" + str(commID) +"'", out_fields="*").df.TermEnds[0])
        if mayor_end == 'None':
            mayor_end_dt = 'no data'
        else:
            mayor_term = str(datetime.utcfromtimestamp(int(mayor_end[0:10])))
            mayor_end_dt = datetime.strftime(datetime.strptime(mayor_term, "%Y-%m-%d %H:%M:%S"), '%m/%d/%Y')
            print(mayor_end_dt)

        #City planning commission
        cp_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/10')
        embed_cp_tbl = officials_tbl(cp_base)  

        #House and senate reps
        hs_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/4')
        reps = hs_base.query(where=house_filter, out_fields="*").df
        house_rep = reps.HouseRepresentative
        senate_rep = reps.Senator
        house_tbl = ['<table><th>House Representatives</th><tbody>']
        #House representative
        for h in set(house_rep):
            h_rep = hs_base.query(where="HouseRepresentative = '" + str(h) + "'", out_fields = "*").df.HouseRepresentative[0]
            house_tbl.append("<tr><td>" + str(h_rep) + "<tr><td>")
        embed_house_rep = ' '.join(house_tbl)
        senate_tbl = ['<table><th>Senators</th><tbody>']
        #Senator
        for s in set(senate_rep):
            senate = hs_base.query(where="Senator = '" + str(s) + "'", out_fields = "*").df.Senator[0]
            senate_tbl.append("<tr><td>" + str(senate) + "<tr><td>")
        embed_senate = ' '.join(senate_tbl)

        #Update the Municipal officials webmap, the content extent
        panel_elect["media"]["webmap"]["id"] = new_webmap
        panel_elect["media"]["webmap"]["extent"]["xmin"] = elect_ext.xmin
        panel_elect["media"]["webmap"]["extent"]["ymin"] = elect_ext.ymin
        panel_elect["media"]["webmap"]["extent"]["xmax"] = elect_ext.xmax
        panel_elect["media"]["webmap"]["extent"]["ymax"] = elect_ext.ymax

        #Print officials tables to the panel text.
        panel_elect["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue"data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p><p>&nbsp;</p><p>' + str(community) + ' is included in House District ' + str(precinct1[0:2]) + ' and Senate District ' + str(senate_dist) + '.</p><p>&nbsp;</p><p>' + str(embed_house_rep) + str(embed_senate) + '</p><p>&nbsp;</p><p>Mayor, End of term: ' + str(mayor) + ', ' + str(mayor_end_dt) + '</p><p>&nbsp;</p><div><p><b><u>City Council:</u></b></p><p>' + str(embed_cc_tbl) + '</tbody></table></p><p>&nbsp;</p><p>&nbsp;</p><div><p><b><u>Borough Assembly:</u></b></p><p>' + str(embed_boro_tbl) + '</p></div>'

        clonedStorymap.save()
        print("Municipal officals data posted to panel.")   
        
### GENERAL SERVICES ###
        #Healthcare tables
        healthcare = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/Services_Healthcare_SafetyNet_Directory/MapServer/0')
        healthcare_fts = healthcare.query(where=filter_ID, out_fields="*").df
        if str(healthcare_fts) == 'Empty DataFrame\nColumns: []\nIndex: []':
            print("This community does not have any healthcare facilities.")
            embed_hlth_tbl = 'There are no healthcare facilities data for this community.'
        else:
            print("Creating healthcare facilities table...")
            tbl_lst = ['<table border="1"><tr><th width="70%">Healthcare Facility</th><th width="30%">Services Provided</th></tr>']
            for h in healthcare_fts.EntityName:
                hc_data = healthcare.query(where="EntityName = '" + str(h) +"'", out_fields = "*").df
                facility = hc_data.EntityName[0]
                services = [hc_data.FacilityType1[0],hc_data.FacilityType2[0],hc_data.FacilityType3[0],hc_data.FacilityType4[0], hc_data.FacilityType5[0]]
                svcs_lst = (str(services).replace(", None", " ")).replace('[',"").replace(']',"").replace("'","")
                tbl_lst.append('<tr><td>' + str(facility) + '</td><td> ' + str(svcs_lst) + '</td></tr>')
            embed_hlth_tbl = ' '.join(tbl_lst)

        #Public libraries
        pl_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Services/MapServer/1')
        if str(pl_base.query(where=query, out_fields="*").df) == 'Empty DataFrame\nColumns: []\nIndex: []':
            print("There are no libraries in " + str(community))
            pub_lib_tbl = "There are no public libraries in " + str(community) + "."
        else:
            libraries = pl_base.query(where=query, out_fields="*").df.EntityName
            pl_tbl = ['<table border="><th>Library Name</th><th>Address</th><th>Phone</th><th>Email</th><tbody>']
            for p in libraries:
                lib = pl_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.EntityName[0]
                lib_address = pl_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Address[0]
                lib_phone = pl_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Phone[0]
                lib_email = pl_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Email[0]
                pl_tbl.append('<tr><td>' + str(lib) + '</td><td>' + str(lib_address) + '</td><td>' + str(lib_phone) + '</td><td>' + str(lib_email) + '</td></tr>')
            pl_tbl.append('</tbody></table>')
            pub_lib_tbl = ' '.join(pl_tbl)

        #Post offices
        post_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Services/MapServer/0')
        if str(post_base.query(where=query,out_fields="*").df) == 'Empty DataFrame\nColumns: []\nIndex: []':
            embed_post = 'There are no post offices operating in ' + str(community) + '.'
        else:
            offices = post_base.query(where=query,out_fields="*").df.EntityName
            post_tbl = ['<table border="1"><th width="30%">Post Office Name</th><th width="20%">Address</th><th width="20%">Phone</th><th width="30%">Email</th><tbody>']
            for p in set(offices):
                post = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.EntityName[0]
                post_address = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Address[0]
                post_phone = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Phone[0]
                post_email = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Email[0]
                post_tbl.append('<tr><td>' + str(post) + '</td><td>' + str(post_address) + '</td><td>' + str(post_phone) + '</td><td>' + str(post_email) + '</td></tr>')
            post_tbl.append('</tbody></table>')
            embed_post = ' '.join(post_tbl)

        #Update the content using tables above
        panel_gensvs["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p><p><i>Enter the name or address of a facility in the search tab to the left to find the service provider on the map.</i></p><p><b>HEALTHCARE:</b></p><div><p>' + str(embed_hlth_tbl) + '</table></p></div><p>&nbsp;</p><p><b>POST OFFICES: </b></p><div>' + str(embed_post) + '</div><p>&nbsp;</p><p><b>PUBLIC LIBRARIES: </b></p><div><p>' + str(pub_lib_tbl) + '</p></div><p>&nbsp;</p><p><b>UTILTIES: (coming soon) </b></p><div></div></p>'

        clonedStorymap.save()
        print("Services tables posted to panel.")



### ANCSA PANEL ###
        #Set the ancsa popup if the community is included in ANCSA 14(c) plat inventory
        plats_lyr = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/ANCSA/ANCSA_14c_Plats/MapServer/0')
        ANCSA_fts = plats_lyr.query(where="COMMUNITY_NAME = '" + str(community) + "'", out_fields="*")
        if str(ANCSA_fts.features) == '[]':
            panel_ANCSA["media"]["webmap"]["popup"] = 'null'
        else:
            point_ft = Geometry(ANCSA_fts.features[0].geometry)
            project = Point({"x":point_ft.x, "y": point_ft.y, "spatialReference": {"wkid":102006}}).project_as(102100)
            ANCSA_oid = ANCSA_fts.df.OBJECTID[0]
            panel_ANCSA["media"]["webmap"]["popup"]["fieldValue"] = int(ANCSA_oid)
            print("ANCSA popup object ID updated to object ID " + str(ANCSA_oid))
            panel_ANCSA["media"]["webmap"]["popup"]["anchorPoint"]["x"] = project.x
            panel_ANCSA["media"]["webmap"]["popup"]["anchorPoint"]["y"] = project.y

        clonedStorymap.save()

        #Update ancsa panel extent using city boundary
        extMain = panel_ANCSA["media"]["webmap"]["extent"]
        extMain["xmin"] = c_ext.xmin
        extMain["xmax"] = c_ext.xmax
        extMain["ymin"] = c_ext.ymin
        extMain["ymax"] = c_ext.ymax

        #Update ancsa panel content
        panel_ANCSA["content"] = "<style type='text/css'>.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class='dcra-blue' data-storymaps='MJ-ACTION-1524783785796' data-storymaps-type='navigate'>Back to Table of Contents</a></p>\n\n<p>&nbsp;</p><p>&nbsp;</p><div><p>The Alaska Native Claims Settlement Act (ANCSA), enacted into law on December 18, 1971, was intended to settle outstanding land claims and establish clear title to Alaska's land and resources. The Act established regional and village corporations. The village corporations received title to the surface estate in and around the village, subject to valid existing rights, as identified in Section 11 of the Act, as amended. Section 14(c)(3) provides that the village corporation shall convey to a municipal corporation (city), or the state in trust, lands identified for present and future community needs. The resources shown below are provided to assist communities with planning for, and carrying out, land conveyances under ANCSA 14(c)(3)</p><p>&nbsp;</p><p><b>" + str(community) + " ANCSA Regional Corporation: </b>" + str(ANCSA_rgn) + "</p></div><p>&nbsp;</p><p>More information about ANCSA 14(c)3 Land Conveyances can be found <a href='http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=24427049328b40cdab001cae38883d7a' target='_blank'>here.</a>&nbsp;</p>\n"
        clonedStorymap.save()
        print("ANCSA data added to panel.")

        
###CONTACTS PANEL###
        #Find Municipality Contact Information
        contacts_lyr = gis.content.get("e8b073f033b74bebbaa365e6dc6b6713")
        contacts_lyr
        contact_info = contacts_lyr.layers[0].query(where="City = '" + str(community) + "'",out_fields='EntityName,Address,City,Zip,Phone,Fax,Website,Email')
        contact_info.df
        #Write municipality info to table
        output = str(contact_info.df)
        if output == 'Empty DataFrame\nColumns: []\nIndex: []':
            print(str(community) + ' is not a municipality. No municipal contact info will be written to panel.')
            panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
        else: 
            def muni_c_lst(info):
                muni_contact = []
                address = info.df.Address[0]
                city = info.df.City[0]
                zipcode = info.df.Zip[0]
                phone1 = info.df.Phone[0]
                phone = str(phone1[0:3] + '-' + phone1[3:6] + '-' + phone1[6:11])
                fax1 = info.df.Fax[0]
                fax = str(fax1[0:3] + '-' + fax1[3:6] + '-' + fax1[6:11])
                website = info.df.Website[0]
                email = info.df.Email[0]
                muni_contact.append("<p style='text-align:left'><u><b>Municipality Contact Information:</b></u></p>")
                muni_contact.append("<p style='text-align:center'><b>" + str(info.df.EntityName[0]) + '</b></p>')
                muni_contact.append("<p style='text-align:center'>" + str(address) + ", " + str(city) + ", AK" + " " + str(zipcode) + '<p>&nbsp;</p>\n\n')
                muni_contact.append("<p style='text-align:center'>Phone Number: " + str(phone) + '</p>')
                muni_contact.append("<p style='text-align:center'>Fax Number: " + str(fax) + '</p><p>&nbsp;</p>\n\n')
                muni_contact.append("<p style='text-align:center'>Website: " + str(website) + '</p>')
                muni_contact.append("<p style='text-align:center'>" + str(email) + '</p><p>&nbsp;</p>\n\n')
                return(' '.join(muni_contact))

            muni_c_lst(contact_info)
            panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div>'+ str(muni_c_lst(contact_info)) + '</div>\n\n<p>&nbsp;</p>\n\n<p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
            clonedStorymap.save()

        #Federally recognized tribe contact info
        tribe_lyr = gis.content.get("c37a47a8196f409aa6cdcf2a2df097bc")
        tribe_contact_info = tribe_lyr.layers[0].query(where="City = '" + str(community) + "'", out_fields='EntityName, Address, City, Zip, Phone, Fax, Website, Email')
        tribe_output = str(tribe_contact_info.df)
        if tribe_output ==  'Empty DataFrame\nColumns: []\nIndex: []':
            print(str(community) + ' is not served by a federally recognized tribe. No FRT data will be written to panel.')
            embed_tribe = str(community) + " is not currently served by a federally recognized tribe."
        else:
            def tribe_info(info):
                tribe_contact = []
                tribe = info.df.EntityName[0]
                tribe_address = info.df.Address[0]
                tribe_city = info.df.City[0]
                tribe_zip = info.df.Zip[0]
                tribe_phone = info.df.Phone[0]
                tribe_fax = info.df.Fax[0]
                tribe_web = info.df.Website[0]
                tribe_email = info.df.Email[0]
                tribe_contact.append("<p style='text-align:left'><u><b>Federally Recognized Tribe Contact Information:</b></u></p>")
                tribe_contact.append("<p style='text-align:center'><b>" + str(info.df.EntityName[0]) + '</b></p>')
                tribe_contact.append("<p style='text-align:center'>" + str(tribe_address) + ", " + str(tribe_city) + ", AK" + " " + str(tribe_zip) + '<p>&nbsp;</p>\n\n')
                tribe_contact.append("<p style='text-align:center'>Phone Number: " + str(tribe_phone) + '</p>')
                tribe_contact.append("<p style='text-align:center'>Fax Number: " + str(tribe_fax) + '</p><p>&nbsp;</p>\n\n')
                tribe_contact.append("<p style='text-align:center'>Website: " + str(tribe_web) + '</p>')
                tribe_contact.append("<p style='text-align:center'>" + str(tribe_email) + '</p><p>&nbsp;</p>\n\n')
                return(' '.join(tribe_contact))
            embed_tribe = tribe_info(tribe_contact_info)
            panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div>'+ str(muni_c_lst(contact_info)) + '</div>\n\n<p>&nbsp;</p>\n\n<div>' + str(embed_tribe) + '</div><p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
            clonedStorymap.save()

        #Native village corporation contact info
        corp_lyr = gis.content.get("2c5868dcf56e4631a155c1a8985c9e26")
        corp_contact_info = corp_lyr.layers[0].query(where=filter_ID, out_fields = 'EntityName, Address, City, State, Zip, Phone, Fax, Website, Email')
        corp_output = str(corp_contact_info.df)
        if corp_output ==  'Empty DataFrame\nColumns: []\nIndex: []':
            print(str(community) + ' is not served by a federally recognized tribe. No Village Regional Corporation data will be written to panel.')
        else:
            def corp_info(info):
                corp_contact = []
                corp = info.df.EntityName[0]
                corp_address = info.df.Address[0]
                corp_city = info.df.City[0]
                corp_zip = info.df.Zip[0]
                corp_phone = info.df.Phone[0]
                corp_fax = info.df.Fax[0]
                corp_web = info.df.Website[0]
                corp_email = info.df.Email[0]
                corp_contact.append("<p style='text-align:left'><u><b>Native Village Corporation Contact Information:</b></u></p>")
                corp_contact.append("<p style='text-align:center'><b>" + str(info.df.EntityName[0]) + '</b></p>')
                corp_contact.append("<p style='text-align:center'>" + str(corp_address) + ", " + str(corp_city) + ", AK" + " " + str(corp_zip) + '<p>&nbsp;</p>\n\n')
                corp_contact.append("<p style='text-align:center'>Phone Number: " + str(corp_phone) + '</p>')
                corp_contact.append("<p style='text-align:center'>Fax Number: " + str(corp_fax) + '</p><p>&nbsp;</p>\n\n')
                corp_contact.append("<p style='text-align:center'>Website: " + str(corp_web) + '</p>')
                corp_contact.append("<p style='text-align:center'>Email: " + str(corp_email) + '</p><p>&nbsp;</p>\n\n')
                return(' '.join(corp_contact))
            embed_corp = corp_info(corp_contact_info)
            panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div>'+ str(muni_c_lst(contact_info)) + '</div>\n\n<p>&nbsp;</p>\n\n<div>' + str(embed_tribe) + '</div><p>\n\n<p>&nbsp;</p>\n\n<div>' + str(embed_corp) + '</div><p>\n\n<p>&nbsp;</p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
            clonedStorymap.save()


### BUSINESS LICENSES ###
        if current_pop < 3000:
            bl_base = gis.content.get("19c770da5bc14c5b9e1bcdb894e1caae")
            licenses = bl_base.layers[0].query(where = "GISS_GIS_OWNER_CDO_Economics_28 = '" + str(identifier) + "' OR GISS_GIS_OWNER_CDO_Economics_29 = '" + str(identifier) + "'", out_fields = '*')
            if str(licenses.df) ==  'Empty DataFrame\nColumns: []\nIndex: []':
                embed_bl_tbl = "There is no avaiable business license data for this community. Use the community search tool at the left to find additional business license information."
            else:
                l_tbl = ['<table border="1"><th>Business Name</th><th>License Number</th><th>Business Type</th><tbody>']
                print("Building business licenses table...")
                for l in licenses.df.GISS_GIS_OWNER_CDO_Economics__2:
                    query = "GISS_GIS_OWNER_CDO_Economics__2 = '" + str(l.replace("'","''")) +"'"
                    l_data = bl_base.layers[0].query(where=query, out_fields="*").df
                    l_number = l_data.GISS_GIS_OWNER_CDO_Economics__1[0]
                    l_name = l_data.GISS_GIS_OWNER_CDO_Economics__2[0]
                    l_expdt = l_data.GISS_GIS_OWNER_CDO_Economics__9[0]
                    l_type = l_data.GISS_GIS_OWNER_CDO_Economics__4[0]
                    l_busname = l_data.GISS_GIS_OWNER_CDO_Economics__6[0]
                    l_cat = l_data.GISS_GIS_OWNER_CDO_Economics_11[0]
                    l_tbl.append('<tr><td>' + str(l_name) + '</td><td>' + str(l_number) + '</td><td>' + str(l_type) + '</td></tr>')
                l_tbl.append('<tbody></table>')
                embed_bl_tbl = ' '.join(l_tbl)

            panel_bl['content'] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1537213770901" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n<p>&nbsp;</p><p> Active business licenses in ' + str(community) + ":</p><p>&nbsp;</p><div>" + str(embed_bl_tbl) + '</div><p>&nbsp;</p><p><i>This data is updated monthly. For the most up-to-date information, use the <b> Alaska Business Licenses search tool to the left </b>, or open the website in a new window: <a href="https://www.commerce.alaska.gov/cbp/main/search/businesses" target = "_blank"> Alaska Division of Corporations, Businesses & Professional Licensing </a></p>'
        else:
            print(str(community) + " has more than 3,000 people. Business licenses table won't be created.")
            panel_bl['content'] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1537213770901" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n<p>&nbsp;</p><p> Alaska DCRA Community Storymaps only provides business license data for communities with over 3,000 people. </p><p>&nbsp;</p><p>&nbsp;</p><p><i>This data is updated monthly. For the most up-to-date information, use the <b> Alaska Business Licenses search tool to the left </b>, or open the website in a new window: <a href="https://www.commerce.alaska.gov/cbp/main/search/businesses" target = "_blank"> Alaska Division of Corporations, Businesses & Professional Licensing </a></p>'
        clonedStorymap.save()

        print("Business licenses posted to panel.")

        #Temporary fix for demography panel popup anchor point
        panel_geog["media"]["webmap"]["popup"]["anchorPoint"]["x"]=project_dcced.x
        panel_geog["media"]["webmap"]["popup"]["anchorPoint"]["y"]=project_dcced.y
        clonedStorymap.save()

        print("Storymap completed for " + str(community))
    except:
        print("ERROR WITH " + str(community) + ". STORYMAP WILL NOT BE CREATED UNTIL ISSUES ARE FIXED.")
        continue
print("All storymaps have been created for municipalities!")

Egegik
Egegik storymap cloned.
Updating table of contents and main picture...
Egegik webmap id: ae572c86b50a41979f2dc058e66bf954
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
ERROR WITH Egegik. STORYMAP WILL NOT BE CREATED UNTIL ISSUES ARE FIXED.
Pilot Station
Pilot Station storymap cloned.
Updating table of contents and main picture...
Pilot Station webmap id: a5345ceb46a14dc8a5afae333f7bfc8d
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
Culture and history panel completed.
Updating climate and geography panel...
Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-18132067.741522282, 8844267.797173519).
Locating census data for Pilot Station...
Census table built.
Building age table...
Building sex table...
Demographics panel updated.
Transportation table posted to panel.
Sc

Creating healthcare facilities table...
There are no libraries in Eek
Services tables posted to panel.
ANCSA popup object ID updated to object ID 104
ANCSA data added to panel.
Building business licenses table...
Business licenses posted to panel.
Storymap completed for Eek
Chefornak
Chefornak storymap cloned.
Updating table of contents and main picture...
Chefornak webmap id: 8d357ec1e87945008abd99c431fa0f0f
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
Culture and history panel completed.
Updating climate and geography panel...
Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-18287603.857705526, 8435002.515864955).
Locating census data for Chefornak...
Census table built.
Building race table...
Building age table...
Building sex table...
Demographics panel updated.
Transportation table posted to panel.
School table pos

Services tables posted to panel.
ANCSA popup object ID updated to object ID 179
ANCSA data added to panel.
Building business licenses table...
Business licenses posted to panel.
Storymap completed for Kaktovik
Nondalton
Nondalton storymap cloned.
Updating table of contents and main picture...
ERROR WITH Nondalton. STORYMAP WILL NOT BE CREATED UNTIL ISSUES ARE FIXED.
Pelican
Pelican storymap cloned.
Updating table of contents and main picture...
Pelican webmap id: ea0d47d854f14d4f98c2ac54e5ebcee9
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
Culture and history panel completed.
Updating climate and geography panel...
Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-15165026.12883012, 7959059.791570503).
Locating census data for Pelican...
Census table built.
Building age table...
Building sex table...
Demographics panel u

ANCSA data added to panel.
Building business licenses table...
Business licenses posted to panel.
Storymap completed for McGrath
Tanana
Tanana storymap cloned.
Updating table of contents and main picture...
Tanana webmap id: b6ad6078009244dfaa408911b7fa13ca
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
Culture and history panel completed.
Updating climate and geography panel...
Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-16929212.503164526, 9653607.035574194).
Locating census data for Tanana...
Census table built.
Building race table...
Building age table...
Building sex table...
Demographics panel updated.
Transportation table posted to panel.
School table posted to the education panel.
Economic data posted to economy panel.
Updating municipal officials content...
City council table built.
No assembly data will be 

Unable to complete operation.


...

There are no libraries in Diomede
Services tables posted to panel.
ANCSA popup object ID updated to object ID 95
ANCSA data added to panel.
Building business licenses table...
Business licenses posted to panel.
Storymap completed for Diomede
False Pass
False Pass storymap cloned.
Updating table of contents and main picture...
False Pass webmap id: 3a3f8d04ef6546deaf242da9a3bdd51f
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
Culture and history panel completed.
Updating climate and geography panel...
Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-18191245.50910541, 7333781.240258523).
Locating census data for False Pass...
Census table built.
Building race table...
Building age table...
Building sex table...
Demographics panel updated.
Transportation table posted to panel.
School table posted to the education panel.
Ec


'where' parameter is invalid


ERROR WITH Clark's Point. STORYMAP WILL NOT BE CREATED UNTIL ISSUES ARE FIXED.
Nuiqsut
Nuiqsut storymap cloned.
Updating table of contents and main picture...
Nuiqsut webmap id: 54f14590a1014fb08c5ae837780973d7
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
Culture and history panel completed.
Updating climate and geography panel...
Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-16809133.595339056, 11139817.571835162).
Locating census data for Nuiqsut...
Census table built.
Building race table...
Building age table...
Building sex table...
Demographics panel updated.
Transportation table posted to panel.
School table posted to the education panel.
Economic data posted to economy panel.
Updating municipal officials content...
City council table built.
10/31/2018
There are no officials in<FeatureLayer url:"http://maps.com

Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-18500504.682817787, 8486293.833891844).
Locating census data for Mekoryuk...
Census table built.
Building race table...
Building age table...
Building sex table...
Demographics panel updated.
Transportation table posted to panel.
School table posted to the education panel.
Economic data posted to economy panel.
Updating municipal officials content...
City council table built.
No assembly data will be added.
10/31/2019
There are no officials in<FeatureLayer url:"http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/10">
Municipal officals data posted to panel.
Creating healthcare facilities table...
There are no libraries in Mekoryuk
Services tables posted to panel.
ANCSA popup object ID updated to object ID 238
ANCSA data added to panel.
Building business licenses table...
Bus

Updating table of contents and main picture...
Koyukuk webmap id: 66980041fe1a44ebba8cffa11a3bbd57
Updating panel extents...
Panel extents updated.
Updating culture and history panel content...
Culture and history panel completed.
Updating climate and geography panel...
Geography and climate info posted to geog/climate panel.
Demographics panel extent updated.
Updating demographics popup...
Demographics popup updated. Anchor point = (-17555384.46633148, 9577302.818487203).
Locating census data for Koyukuk...
Census table built.
Building race table...
Building age table...
Building sex table...
Demographics panel updated.
Transportation table posted to panel.
School table posted to the education panel.
Economic data posted to economy panel.
Updating municipal officials content...
City council table built.
No assembly data will be added.
10/31/2018
There are no officials in<FeatureLayer url:"http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServe